In [ ]:
from getdist import plots, loadMCSamples, MCSamples
import numpy as np
%matplotlib inline
from tabulate import tabulate
import matplotlib.pyplot as plt
plt.rcParams.update({"text.usetex": True,"font.family": "serif"})
import matplotlib as mpl

import plotTools as pt

In [ ]:
cosmoParamsChain = ['omch2', 'ombh2', 'nnu', 'As', 'ns', 'tau', 'thetastar', 'mnu']

In [ ]:
##########
### Include the provided ".paramnames" files in your MCMC output directory
### if you want the parameters to have nice labels in the resulting figure.
##########

In [ ]:
jobName = './results/DALI_ExpB_60meV'
expNumber = 0
fsky = 0.6
fishers, cosmoParams = pt.loadWithDALI(jobName = jobName, pythonFlag = 3, returnCosmoParams = True, gTypes=['Gaussian'])
fishers = pt.addfskyWithDALI(fishers,fsky, gTypes=['Gaussian'])
fishers = pt.addTauWithDALI(fishers, cosmoParams, gTypes=['Gaussian'])
covs = pt.invertFishers(fishers, gTypes=['Gaussian'])
sigmas = pt.getSigmas(covs, gTypes=['Gaussian'])

samples = loadMCSamples('./chains/CMB_ExpB60meV', settings={'ignore_rows':0.3})
samplesDALI = loadMCSamples('./chains/DALI_lensed_ExpB60meV', settings={'ignore_rows':0.3})

In [ ]:
renameParams = dict()
for i in range(0,8):
    renameParams[cosmoParams[i]] = cosmoParamsChain[i]
    
samplesDALI.updateRenames(renameParams)

In [ ]:
cov_all = dict()
cov_all['parameters'] = cosmoParams
cov_all['CMBlike means'] = samples.mean(cosmoParamsChain)
cov_all['fiducial values'] = np.array([0.1197,0.0222,3.046,2.196e-9,0.9655,0.06,0.010409,0.06])
cov_all['Fisher covs'] = sigmas['Gaussian']['lensed'][expNumber]
cov_all['DALI covs'] = np.sqrt(np.diag(samplesDALI.cov(cosmoParamsChain)))
cov_all['CMBlike covs'] = np.sqrt(np.diag(samples.cov(cosmoParamsChain)))

In [ ]:
from getdist.gaussian_mixtures import GaussianND

analytic_lensed_g = GaussianND(cov_all['fiducial values'],\
                      covs['Gaussian']['lensed'][expNumber],\
                      names = cosmoParamsChain,\
                      labels = cosmoParamsChain,\
                      label = 'analytic Fisher')

In [ ]:
print(tabulate(cov_all,headers=cov_all.keys(),tablefmt="github"))

In [ ]:
myColors = ['C1', 'C2', 'C4']

In [ ]:
### Curvature calculation
"""
@author: Joe Ryan and Cynthia Trendafilova
"""
from numpy import log10, loadtxt, savetxt, sqrt, mgrid, argwhere, floor, c_, arange, linspace, reshape
from numpy.linalg import inv
from sympy import symbols, diff, lambdify
import matplotlib.pyplot as plt
from itertools import combinations
import pickle

jobName = './results/DALI_ExpB_60meV'
fid_mnu = 0.06
mnu_lab = 'lensed_60_meV'
print(mnu_lab)

with open(jobName + ".pkl", 'rb') as f:
    data = pickle.load(f, encoding="latin1")

data.keys()

### Gaussian Fisher results - lensed
F = data['fisherGaussian'][0]['lensed']

### Gaussian DALI3/4 results (tensors for doublet DALI) - lensed
G = data['DALI3Gaussian'][0]['lensed']
H = data['DALI4Gaussian'][0]['lensed']

Slab = 'ExpB'
print(Slab)

fid_omch2 = 0.1197
fid_ombh2 = 0.0222
fid_nnu = 3.046
#fid_As = 2.196*(10**(-9))
fid_As = 2.196
fid_ns = 0.9655
fid_tau = 0.06
fid_thetastar = 0.010409

plotlab = ['oc2', 'ob2', 'n', 'A', 'ns', 't', 'th', 'm']
plotleglab = ['$\Omega_{c}h^2$', '$\Omega_{b}h^2$', '$N_{\\rm eff}$', '$A_s$', '$n_s$', '${\\tau}$', '${\\theta}_{s}$', '$m_{\\nu}$']

stepL = [0.116, 0.0220, 2.95, 2, 0.95, 0.03, 0.010405, -0.2]
stepR = [0.124, 0.0224, 3.20, 2.4, 0.98, 0.09, 0.010415, 0.3]

x = symbols('x', real=True)
y = symbols('y', real=True)

P = list(combinations(range(0, 8), r=2))

#CT
negativePoints = dict()
for q in range(0,8):
    negativePoints[plotlab[q]] = dict()

for i in range(0,8):
    for j in range(0,8):
        if i<j:
            p_fid = [fid_omch2, fid_ombh2, fid_nnu, fid_As, fid_ns, fid_tau, fid_thetastar, fid_mnu]

            ij = [i, j]
            print(i, j, 'V7.5')

            p = [x - p_fid[i], y - p_fid[j]]

            #CT
            negativePoints[plotlab[i]][plotlab[j]] = dict()

            if i == 3:
                p[0] = (10**(-9))*p[0]
            if j == 3:
                p[1] = (10**(-9))*p[1]

            sumF = 0.6*(-1/2)*(F[i][i]*p[0]*p[0] + F[i][j]*p[0]*p[1] + F[j][i]*p[1]*p[0] + F[j][j]*p[1]*p[1])
            sumG = 0
            sumH = 0

            for a in range(0, 2, 1):
                for b in range(0, 2, 1):
                    for c in range(0, 2, 1):
                        sumG += 0.6*(-1/2)*G[ij[a]][ij[b]][ij[c]]*p[a]*p[b]*p[c]

            for a in range(0, 2, 1):
                for b in range(0, 2, 1):
                    for c in range(0, 2, 1):
                        for d in range(0, 2, 1):
                            sumH += 0.6*(-1/8)*H[ij[a]][ij[b]][ij[c]][ij[d]]*p[a]*p[b]*p[c]*p[d]

            Phi = -(sumF + sumG + sumH)

            Phi0 = diff(Phi, x)
            Phi1 = diff(Phi, y)

            Phi00 = diff(Phi0, x)
            Phi01 = diff(Phi0, y)
            Phi11 = diff(Phi1, y)

            k = (Phi00*(Phi1**2) + Phi11*(Phi0**2) - 2*Phi1*Phi01*Phi0)/((Phi0**2 + Phi1**2)**(3/2))
            K = lambdify([x, y], k)

            x0, x1 = mgrid[stepL[i]:stepR[i]:16j, stepL[j]:stepR[j]:16j]

            ### pass to lambdify'd functions
            ### output is 2d array of curvatures
            K_D = K(x0, x1)

            ### indices of negative values in DALI
            negativeD = argwhere(K_D <= 0)

            ### Sanity check:
            Phi_F = -sumF

            Phi0 = diff(Phi_F, x)
            Phi1 = diff(Phi_F, y)

            Phi00 = diff(Phi0, x)
            Phi01 = diff(Phi0, y)
            Phi11 = diff(Phi1, y)

            k = (Phi00*(Phi1**2) + Phi11*(Phi0**2) - 2*Phi1*Phi01*Phi0)/((Phi0**2 + Phi1**2)**(3/2))
            K = lambdify([x, y], k)

            x0, x1 = mgrid[stepL[i]:stepR[i]:16j, stepL[j]:stepR[j]:16j]

            ### pass to lambdify'd functions
            ### output is 2d array of curvatures
            K_F = K(x0, x1)

            ### indices of negative values in DALI
            negativeF = argwhere(K_F <= 0)

            X0 = []
            X1 = []

            for u in arange(0, len(negativeD), 1):
                I0 = negativeD[u][0]
                I1 = negativeD[u][1]

                X0.append(x0[I0][I1])
                X1.append(x1[I0][I1])

            ### Plot coordinates at which K <= 0 over Fisher and Doublet cross-section contours:
            X = linspace(stepL[i], stepR[i], 50)
            Y = linspace(stepL[j], stepR[j], 50)

            Z = []
            Z_F = []
            xl = []
            yl = []

            Phi_L = lambdify([x, y], Phi)
            Phi_FL = lambdify([x, y], Phi_F)
            for xc0 in X:
                for xc1 in Y:
                    Z.append(Phi_L(xc0, xc1))
                    Z_F.append(Phi_FL(xc0, xc1))
                    xl.append(xc0)
                    yl.append(xc1)

            x_2d = reshape(xl, (50, 50))
            y_2d = reshape(yl, (50, 50))
            z_2d = reshape(Z, (50, 50))
            zf_2d = reshape(Z_F, (50, 50))

            #CT
            if plotlab[i]=='A':
                x_2d*=(10**(-9))
                X0 = (np.array(X0)*(10**(-9))).tolist()
            if plotlab[j]=='A':
                y_2d*=(10**(-9))
                X1 = (np.array(X1)*(10**(-9))).tolist()

            #CT
            negativePoints[plotlab[i]][plotlab[j]]['x'] = X0
            negativePoints[plotlab[i]][plotlab[j]]['y'] = X1

            #CT
            negativePoints[plotlab[i]][plotlab[j]]['x2'] = x_2d
            negativePoints[plotlab[i]][plotlab[j]]['y2'] = y_2d
            negativePoints[plotlab[i]][plotlab[j]]['f2'] = zf_2d
            negativePoints[plotlab[i]][plotlab[j]]['d2'] = z_2d

In [ ]:
g1 = plots.get_subplot_plotter()

g1.settings.axes_labelsize=36
g1.settings.axes_fontsize=26
g1.settings.legend_fontsize = 30

g1.settings.line_labels = False

In [ ]:
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

legend_elements = [Patch(facecolor=myColors[0], edgecolor=myColors[0], label = 'Mock Likelihood'),
                  Patch(facecolor=myColors[1], edgecolor=myColors[1], label = 'DALI'),
                  Line2D([0], [0], color=myColors[2], lw=2, label = 'Fisher')]

legend_elements_upper = [Line2D([0], [0], marker='o', color='w', label = 'Concavity', markerfacecolor='C3', markersize = 10),
                         Line2D([0], [0], color=myColors[1], lw=2, label = 'DALI', linestyle='dashed'),
                         Line2D([0], [0], color=myColors[2], lw=2, label = 'Fisher', linestyle='dotted')]

In [ ]:
bPlot = g1.triangle_plot([samples,samplesDALI,analytic_lensed_g],\
                cosmoParamsChain,\
                filled=[True,True,False],\
                colors=myColors,contour_colors=myColors, \
                contour_lws=[1,1,2], \
                        upper_roots = [samplesDALI],
                        upper_kwargs = {'colors':['w'],'contour_colors':['w'],'contour_lws':[0],'filled':[False]},
                        upper_label_right = True)

In [ ]:
myTitleFont = mpl.font_manager.FontProperties(weight='bold',size=30)

legend1 = g1.fig.legend(handles=legend_elements, fontsize=30, ncol=3,\
             bbox_to_anchor=(0.5,-0.05), loc='upper center', borderaxespad=0)
legend1.set_title(r'\textbf{Marginalized likelihood contours}',prop=myTitleFont)

legend2 = g1.fig.legend(handles=legend_elements_upper, fontsize=30, ncol=3,\
             bbox_to_anchor=(0.5,1.00), loc='lower center', borderaxespad=0)
legend2.set_title(r'\textbf{Likelihood cross-sections}',prop=myTitleFont)

In [ ]:
jcount = 0

for i in range(0,8):
    for j in range(0,i):
        jcount+=1
        
        g1.fig.get_axes()[-(i+jcount)].contour(negativePoints[plotlab[-(i+1)]][plotlab[-i+j]]['y2'],negativePoints[plotlab[-(i+1)]][plotlab[-i+j]]['x2'],\
                                            negativePoints[plotlab[-(i+1)]][plotlab[-i+j]]['d2'],[10,50,100],\
                                            colors=myColors[1], linestyles='dashed', linewidths=2)
        g1.fig.get_axes()[-(i+jcount)].contour(negativePoints[plotlab[-(i+1)]][plotlab[-i+j]]['y2'],negativePoints[plotlab[-(i+1)]][plotlab[-i+j]]['x2'],\
                                            negativePoints[plotlab[-(i+1)]][plotlab[-i+j]]['f2'],[10,50,100],\
                                            colors=myColors[2], linestyles='dotted', linewidths=2)
        
        g1.fig.get_axes()[-(i+jcount)].scatter(negativePoints[plotlab[-(i+1)]][plotlab[-i+j]]['y'],negativePoints[plotlab[-(i+1)]][plotlab[-i+j]]['x'],color='C3',zorder=3)
        xvals,yvals = g1.fig.get_axes()[-(i+jcount)].get_xlim(),g1.fig.get_axes()[-(i+jcount)].get_ylim()
        xrange = xvals[1]-xvals[0]
        yrange = yvals[1]-yvals[0]
        g1.fig.get_axes()[-(i+jcount)].set_aspect(1.*(xrange/yrange), adjustable='box')
    jcount-=1

for i in range(0,28+8):
    xvals,yvals = g1.fig.get_axes()[i].get_xlim(),g1.fig.get_axes()[i].get_ylim()
    xrange = xvals[1]-xvals[0]
    yrange = yvals[1]-yvals[0]
    g1.fig.get_axes()[i].set_aspect(1.*(xrange/yrange), adjustable='box')

In [ ]:
g1.fig.get_axes()[-(1)].spines["bottom"].set_linewidth(4)
g1.fig.get_axes()[-(1)].spines["left"].set_linewidth(4)

g1.fig.get_axes()[-(2)].spines["bottom"].set_linewidth(4)
g1.fig.get_axes()[-(2)].spines["left"].set_linewidth(4)

g1.fig.get_axes()[-(4)].spines["bottom"].set_linewidth(4)
g1.fig.get_axes()[-(4)].spines["left"].set_linewidth(4)

g1.fig.get_axes()[-(7)].spines["bottom"].set_linewidth(4)
g1.fig.get_axes()[-(7)].spines["left"].set_linewidth(4)

g1.fig.get_axes()[-(11)].spines["bottom"].set_linewidth(4)
g1.fig.get_axes()[-(11)].spines["left"].set_linewidth(4)

g1.fig.get_axes()[-(16)].spines["bottom"].set_linewidth(4)
g1.fig.get_axes()[-(16)].spines["left"].set_linewidth(4)

g1.fig.get_axes()[-(22)].spines["bottom"].set_linewidth(4)
g1.fig.get_axes()[-(22)].spines["left"].set_linewidth(4)

In [ ]:
#omc
g1.fig.get_axes()[0].xaxis.set_major_locator(plt.FixedLocator([0.118,0.122]))
g1.fig.get_axes()[0].set_xticklabels([0.118,0.122],rotation = 45)
#mnu
g1.fig.get_axes()[0].yaxis.set_major_locator(plt.FixedLocator([0.1,0.2]))
g1.fig.get_axes()[0].set_yticklabels([100,200])

#omb
g1.fig.get_axes()[8].xaxis.set_major_locator(plt.FixedLocator([0.02212,0.02228]))
g1.fig.get_axes()[8].set_xticklabels([0.02212,0.02228],rotation = 45)

#Neff
g1.fig.get_axes()[15].xaxis.set_major_locator(plt.FixedLocator([3.0,3.1]))
g1.fig.get_axes()[15].set_xticklabels([3.0,3.1],rotation=45)

#As
g1.fig.get_axes()[21].annotate(r'$\times$10$^{-9}$',
                             xy=(.60, -.40), xycoords='axes fraction', fontsize=18)
g1.fig.get_axes()[21].xaxis.set_major_locator(plt.FixedLocator([2.15*10**(-9),2.25*10**(-9)]))
g1.fig.get_axes()[21].set_xticklabels([2.15,2.25],rotation=45)

#ns
g1.fig.get_axes()[26].xaxis.set_major_locator(plt.FixedLocator([0.96,0.97]))
g1.fig.get_axes()[26].set_xticklabels([0.96,0.97],rotation=45)

#tau
g1.fig.get_axes()[30].xaxis.set_major_locator(plt.FixedLocator([0.05,0.07]))
g1.fig.get_axes()[30].set_xticklabels([0.05,0.07],rotation=45)

#thetas
g1.fig.get_axes()[33].xaxis.set_major_locator(plt.FixedLocator([0.010407,0.010411]))
g1.fig.get_axes()[33].set_xticklabels([0.010407,0.010411],rotation=45)

#mnu
g1.fig.get_axes()[35].xaxis.set_major_locator(plt.FixedLocator([0.1,0.2]))
g1.fig.get_axes()[35].set_xticklabels([100,200],rotation=45)

In [ ]:
# correct ratio seems to be 16 x 13
rescaleFactor = 1.25
g1.fig.set_size_inches(16*rescaleFactor,13*rescaleFactor)

In [ ]:
g1.fig

In [ ]:
g1.fig.savefig('ExpB_CMB_60meV.pdf',bbox_inches='tight',pad_inches=0.3)